In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score 
from sklearn import preprocessing 

In [3]:
data = pd.read_csv('.\kag_risk_factors_cervical_cancer.csv')
data.head()

<>:1: SyntaxWarning: invalid escape sequence '\k'
<>:1: SyntaxWarning: invalid escape sequence '\k'
C:\Users\ayush\AppData\Local\Temp\ipykernel_18796\18677860.py:1: SyntaxWarning: invalid escape sequence '\k'
  data = pd.read_csv('.\kag_risk_factors_cervical_cancer.csv')


,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,?,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,?,?,0,0,0,0,0,0,0,0


In [4]:
data['STDs'].value_counts()

STDs
0.0    674
?      105
1.0     79
Name: count, dtype: int64

In [5]:
data = data.drop(columns = ['Hinselmann', 'Schiller', 'Citology', 'Dx', 'STDs:condylomatosis', 'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis', 'STDs:molluscum contagiosum', 'STDs:vulvo-perineal condylomatosis', 'STDs:pelvic inflammatory disease', 'STDs:condylomatosis', 'STDs:cervical condylomatosis'], axis = 1)

In [6]:
data = data.replace('?', np.nan)
data.isnull().sum()

Age                                   0
Number of sexual partners            26
First sexual intercourse              7
Num of pregnancies                   56
Smokes                               13
Smokes (years)                       13
Smokes (packs/year)                  13
Hormonal Contraceptives             108
Hormonal Contraceptives (years)     108
IUD                                 117
IUD (years)                         117
STDs                                105
STDs (number)                       105
STDs:syphilis                       105
STDs:genital herpes                 105
STDs:AIDS                           105
STDs:HIV                            105
STDs:Hepatitis B                    105
STDs:HPV                            105
STDs: Number of diagnosis             0
STDs: Time since first diagnosis    787
STDs: Time since last diagnosis     787
Dx:Cancer                             0
Dx:CIN                                0
Dx:HPV                                0


In [7]:
binary = ['IUD', 'IUD (years)', 'Smokes', 'Smokes (years)', 'STDs:HPV', 'STDs:Hepatitis B', 'STDs:HIV', 'STDs:AIDS', 'STDs:genital herpes', 'STDs:syphilis', 'Hormonal Contraceptives', 'Hormonal Contraceptives (years)', 'STDs', 'STDs (number)']
for i in binary:
    data[i].fillna(0, inplace = True)

columns = ['Number of sexual partners', 'First sexual intercourse', 'Smokes (packs/year)', 'Num of pregnancies', 'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis']
for i in columns:
    data[i] = pd.to_numeric(data[i], errors='coerce')  # Coerce errors to NaN
    data[i].fillna(data[i].median(), inplace=True)



C:\Users\ayush\AppData\Local\Temp\ipykernel_18796\2492810848.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[i].fillna(0, inplace = True)
C:\Users\ayush\AppData\Local\Temp\ipykernel_18796\2492810848.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doin

In [11]:
data = data.to_csv('cleaned.csv', index = False)

In [8]:
X = data.drop(columns = ['Biopsy'], axis = 1)
y = data['Biopsy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, stratify = y, random_state = 69)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

models = []
models.append(LogisticRegression(max_iter=1000))
models.append(SVC(max_iter=1000))
models.append(GaussianNB())
models.append(KNeighborsClassifier())
models.append(DecisionTreeClassifier())

In [10]:
results = []
accuracies = []

for model in models:
    model = model.fit(X_train, y_train)
    pred = model.predict(X_test)
    score = cross_val_score(model, X_train, y_train, cv = 4)
    results.append(score.mean())
    accuracies.append(accuracy_score(pred, y_test))

optimal = max(accuracies)
i = accuracies.index(optimal)
print("Optimal Model: ", models[i])
print("Accuracy: ", accuracies[i])
print("Cross Validation score: ", results[i])

C:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Optimal Model:  SVC(max_iter=1000)
Accuracy:  0.937984496124031
Cross Validation score:  0.9355296343001263


In [15]:
final_model = models[i]
final_model.fit(X_train, y_train)
import pickle
with open('thyroid.pkl', 'wb') as f:
  pickle.dump(final_model, f)

C:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
